In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Mounting Drive**

In [2]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

Loading Data from Drive

In [3]:
train_data=pd.read_csv(r'/content/drive/MyDrive/Project_smile_classification/train.csv',header=None)
test_data=pd.read_csv(r'/content/drive/MyDrive/Project_smile_classification/test.csv',header=None)
len(train_data),len(test_data)

(4831, 1611)

Data Visualisation

In [4]:
train_data.head()

,0,1
0,05684c70-113d-4894-920e-140edf323528,negative smile
1,0403faa2-2d45-4779-b40a-794c220d3855,positive smile
2,1ba6784e-3684-4f76-81b7-504610a497e1,positive smile
3,675c87fb-bf50-481e-949d-07b5dc14b589,positive smile
4,02567fbb-1147-4186-8d7d-38611569fc06,NOT smile


In [5]:
test_data.head()

,0,1
0,7a0e7396-47ca-421c-84bc-de7c946fc163,NOT smile
1,1e53ec28-b4c0-4eef-ad91-4940f535d354,NOT smile
2,05f0d520-8e85-4490-a160-5a346fc21fd3,positive smile
3,0adf2567-b325-47a8-a2c9-83b34c021163,NOT smile
4,0df7e500-a1d0-44dd-923d-25809fa38611,NOT smile


In [6]:
train_data.columns = ["id_name", "smile_class"]
test_data.columns = ["id_name", "smile_class"]

In [7]:
def append_jpg(s):
  return s+".jpg"

col=list(train_data["id_name"])
mod_col=[]
for s in col:
    mod_col.append(append_jpg(s))
train_data["id_name"]=mod_col


col2=list(test_data["id_name"])
mod_col2=[]
for s in col2:
    mod_col2.append(append_jpg(s))
test_data["id_name"]=mod_col2

In [8]:
train_data.head()

,id_name,smile_class
0,05684c70-113d-4894-920e-140edf323528.jpg,negative smile
1,0403faa2-2d45-4779-b40a-794c220d3855.jpg,positive smile
2,1ba6784e-3684-4f76-81b7-504610a497e1.jpg,positive smile
3,675c87fb-bf50-481e-949d-07b5dc14b589.jpg,positive smile
4,02567fbb-1147-4186-8d7d-38611569fc06.jpg,NOT smile


In [9]:
train_data.shape

(4831, 2)

In [10]:
test_data.head()

,id_name,smile_class
0,7a0e7396-47ca-421c-84bc-de7c946fc163.jpg,NOT smile
1,1e53ec28-b4c0-4eef-ad91-4940f535d354.jpg,NOT smile
2,05f0d520-8e85-4490-a160-5a346fc21fd3.jpg,positive smile
3,0adf2567-b325-47a8-a2c9-83b34c021163.jpg,NOT smile
4,0df7e500-a1d0-44dd-923d-25809fa38611.jpg,NOT smile


In [11]:
test_data.shape

(1611, 2)

In [12]:
class_site=set(train_data["smile_class"])
print(class_site)
len(class_site)

{'NOT smile', 'positive smile', 'negative smile'}


3

In [13]:
train_data.shape

(4831, 2)

In [14]:
test_data.shape

(1611, 2)

In [15]:
train_data.info(),test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831 entries, 0 to 4830
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_name      4831 non-null   object
 1   smile_class  4831 non-null   object
dtypes: object(2)
memory usage: 75.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1611 entries, 0 to 1610
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_name      1611 non-null   object
 1   smile_class  1611 non-null   object
dtypes: object(2)
memory usage: 25.3+ KB


(None, None)

** Creating generators to training model in a batch of images simultanously **

In [16]:
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import keras
from keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten, BatchNormalization
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.10)

In [17]:
train_generator=datagen.flow_from_dataframe(
dataframe=train_data,
directory="/content/drive/MyDrive/Project_smile_classification/happy_images/",
x_col="id_name",
y_col="smile_class",
subset="training",
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 4348 validated image filenames belonging to 3 classes.


In [18]:
valid_generator=datagen.flow_from_dataframe(
dataframe=train_data,
directory=r"/content/drive/MyDrive/Project_smile_classification/happy_images/",
x_col="id_name",
y_col="smile_class",
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))


Found 483 validated image filenames belonging to 3 classes.


In [19]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_data,
directory="/content/drive/MyDrive/Project_smile_classification/happy_images/",
x_col="id_name",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 1611 validated image filenames.


In [20]:
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers.merge import concatenate
import keras
from keras.regularizers import l2
from ipykernel import kernelapp as app
from keras import regularizers, optimizers
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten, BatchNormalization


**CNN Model**

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

**Training CNN Model**

In [23]:

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
history=model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=25
)

Epoch 1/25
271/271 [==============================] - 2472s 9s/step - loss: 0.8368 - accuracy: 0.6928 - val_loss: 0.8300 - val_accuracy: 0.6938
Epoch 2/25
271/271 [==============================] - 30s 111ms/step - loss: 0.8169 - accuracy: 0.6944 - val_loss: 0.8124 - val_accuracy: 0.6917
Epoch 3/25
271/271 [==============================] - 30s 111ms/step - loss: 0.8004 - accuracy: 0.6948 - val_loss: 0.8003 - val_accuracy: 0.6917
Epoch 4/25
271/271 [==============================] - 30s 112ms/step - loss: 0.7745 - accuracy: 0.6948 - val_loss: 0.7831 - val_accuracy: 0.7000
Epoch 5/25
271/271 [==============================] - 30s 112ms/step - loss: 0.7152 - accuracy: 0.7198 - val_loss: 0.7359 - val_accuracy: 0.7563
Epoch 6/25
271/271 [==============================] - 30s 112ms/step - loss: 0.6791 - accuracy: 0.7419 - val_loss: 0.7015 - val_accuracy: 0.7563
Epoch 7/25
271/271 [==============================] - 30s 112ms/step - loss: 0.6593 - accuracy: 0.7447 - val_loss: 0.6998 - val_acc

**Making Predictions based on what Model has Learned**

In [24]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


1611/1611 [==============================] - 833s 517ms/step


In [25]:
type(pred)

numpy.ndarray

In [26]:
print(pred)

[[9.9890101e-01 8.8388781e-04 2.1506063e-04]
 [9.9387425e-01 4.4217240e-03 1.7040122e-03]
 [2.6020429e-03 3.3785827e-02 9.6361214e-01]
 ...
 [9.9834883e-01 1.0642251e-03 5.8700476e-04]
 [9.9996948e-01 2.7601600e-05 2.8735672e-06]
 [9.6155846e-01 6.9781751e-03 3.1463377e-02]]


**Predicting the class with Maximum Probability**

In [27]:
# y_pred_out=[]
for row in pred:
  mx=np.max(row)
  k=0
  for i in range(0,3):
    if(row[i]==mx):
      row[i]=1
    else:
      row[i]=0


pred[0:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [28]:
not_smile=0
positive=0
negative=0
none=0

for row in pred:
  if(row[0]==1.0):
    not_smile=not_smile+1
  if(row[1]==1.0):
    negative=negative+1
  if(row[2]==1.0):
    positive=positive+1
# due to incompetible batch-size  
not_smile,positive,negative

(1335, 241, 35)

**Labeling the classes depending upon the predictions to evaluate Confusion matrix**

In [29]:
y_pred=[]

for row in pred:
  if(row[0]==1.0):
    y_pred.append("NOT smile")
  if(row[1]==1.0):
    y_pred.append("negative smile")
  if(row[2]==1.0):
    y_pred.append("positive smile")

In [30]:
y_pred=np.array(y_pred)
y_pred

array(['NOT smile', 'NOT smile', 'positive smile', ..., 'NOT smile',
       'NOT smile', 'NOT smile'], dtype='<U14')

In [31]:
y_act=test_data["smile_class"]
y_act=np.array(y_act)
y_act

array(['NOT smile', 'NOT smile', 'positive smile', ..., 'NOT smile',
       'NOT smile', 'NOT smile'], dtype=object)

In [32]:
len(y_act),len(y_pred)

(1611, 1611)

**Confusion Matrix on true_values VS predicted values**

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_act, y_pred)

array([[1073,   10,   45],
       [ 104,    8,   31],
       [ 158,   17,  165]])

**Accuracy_Score on true_values VS predicted values**

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_act, y_pred)

0.7734326505276226

**Saving the predicted values in local system**

In [ ]:
# filenames=test_generator.filenames
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":y_pred})
# print(results)
# # results.to_csv("results.csv",index=False)